In [1]:
import pandas as pd
import calendar
import numpy as np
from datetime import date, timedelta

In [2]:
df_cpt = pd.DataFrame({'cpt_id':[100,101,102,103],
                       'freq_of_pmt':['Monthly','Quaterly','Half_yearly','Yearly']})
df_cpt

,cpt_id,freq_of_pmt
0,100,Monthly
1,101,Quaterly
2,102,Half_yearly
3,103,Yearly


In [3]:
data = {
    'CONTRACT_ID': [12345, 12879, 12987, 12346, 12348],
    'CONTRACT_TYPE': ['government', 'individual', 'individual', 'government', 'government'],
    'CONT_S_DATE': ['10-Jan-10', '03-FEB-11', '15-Feb-11', '13-MAR-09', '6-Apr-12'],
    'CONT_E_DATE': ['10-JAN-12', '15-Feb-12', '15-SEPT-11', '14-MAR-11', '22-May-13'],
    'CPT_ID': [100, 101, 100, 102, 103, ],
    'CONT_AMT': [700000, 489938, 289456, 200000, 200000],
    'INS_DATE': ['10-Jan-10', '3-Feb-11', '3-Feb-11', '13-Mar-09', '6-Apr-12'],
    'UPD_DATE': [np.nan, np.nan, '10-Apr-09', np.nan,np.nan ]
}


df_contract = pd.DataFrame(data)
df_contract['CONT_S_DATE'] = pd.to_datetime(df_contract['CONT_S_DATE'])
df_contract['CONT_E_DATE'] = pd.to_datetime(df_contract['CONT_E_DATE'])
df_contract['INS_DATE'] = pd.to_datetime(df_contract['INS_DATE'])
df_contract['UPD_DATE'] = pd.to_datetime(df_contract['UPD_DATE'])
df_contract

C:\Users\HP\AppData\Local\Temp\ipykernel_11644\2160892369.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_contract['CONT_S_DATE'] = pd.to_datetime(df_contract['CONT_S_DATE'])
C:\Users\HP\AppData\Local\Temp\ipykernel_11644\2160892369.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_contract['CONT_E_DATE'] = pd.to_datetime(df_contract['CONT_E_DATE'])
C:\Users\HP\AppData\Local\Temp\ipykernel_11644\2160892369.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_contract['INS_DATE'] = pd.to_datetime(df_contract['INS_DATE'])


,CONTRACT_ID,CONTRACT_TYPE,CONT_S_DATE,CONT_E_DATE,CPT_ID,CONT_AMT,INS_DATE,UPD_DATE
0,12345,government,2010-01-10,2012-01-10,100,700000,2010-01-10,NaT
1,12879,individual,2011-02-03,2012-02-15,101,489938,2011-02-03,NaT
2,12987,individual,2011-02-15,2011-09-15,100,289456,2011-02-03,2009-04-10
3,12346,government,2009-03-13,2011-03-14,102,200000,2009-03-13,NaT
4,12348,government,2012-04-06,2013-05-22,103,200000,2012-04-06,NaT


In [52]:

result=pd.DataFrame()
for index,rows in df_contract.iterrows():
    s_date=rows['CONT_S_DATE']
    e_date=rows['CONT_E_DATE']
    due=rows['CPT_ID']
    amt=rows['CONT_AMT']
    id=rows['CONTRACT_ID']
    day=rows.INS_DATE.day
    mon=rows.INS_DATE.month
    df=pd.DataFrame()
    re=pd.DataFrame()
    date_range = pd.date_range(s_date, e_date - timedelta(days = 1), freq='d')
    if due==100:
        df['date']=date_range
        date=df.loc[df['date'].dt.day == day].reset_index(drop=True) 
        re['date']=date
        installment=len(date)
        inst_amt=round(amt/installment)
        re['amt']=inst_amt
        re['cont_id']=id
        result=pd.concat([result,re])
    elif due==101:
        df['date']=date_range
        date=df.loc[df['date'].dt.day == day].reset_index(drop=True)
        d=date.loc[(date['date'].dt.month== mon)|(date['date'].dt.month== mon+3)|(date['date'].dt.month== mon+6)|(date['date'].dt.month== mon+9)].reset_index(drop=True)
        re['date']=d
        installment=len(d)
        inst_amt=amt/installment
        re['amt']=inst_amt
        re['cont_id']=id
        result=pd.concat([result,re])
    elif due==102:
        df['date']=date_range
        date=df.loc[df['date'].dt.day == day].reset_index(drop=True)
        d=date.loc[(date['date'].dt.month== mon)|(date['date'].dt.month== mon+3)].reset_index(drop=True)
        installment=len(d)
        inst_amt=amt/installment
        re['date']=d
        re['amt']=inst_amt
        re['cont_id']=id
        result=pd.concat([result,re])
    elif due==103:
        df['date']=date_range
        res=df.loc[df['date'].dt.day == day].reset_index(drop=True)
        res=res.loc[res['date'].dt.month == mon].reset_index(drop=True)
        installment=len(res)
        inst_amt=amt/installment
        re['date']=res
        re['amt']=inst_amt
        re['cont_id']=id
        result=pd.concat([result,re])
    
    else:
        continue

In [53]:
result['exp_rec_id']=range(11,11+len(result['date']))
result=result.reset_index(drop=True)
result = result.iloc[:, [3,0, 2, 1]] 
result.to_excel('contract_installment.xlsx',index=False)

result

,exp_rec_id,date,cont_id,amt
0,11,2010-01-10,12345,29167.0
1,12,2010-02-10,12345,29167.0
2,13,2010-03-10,12345,29167.0
3,14,2010-04-10,12345,29167.0
4,15,2010-05-10,12345,29167.0
5,16,2010-06-10,12345,29167.0
6,17,2010-07-10,12345,29167.0
7,18,2010-08-10,12345,29167.0
8,19,2010-09-10,12345,29167.0
9,20,2010-10-10,12345,29167.0
